

>**INTENT CLASSIFICATION USING BERT**




In [0]:
!nvidia-smi

Sun May  3 18:08:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install tensorflow-gpu >> /dev/null

ERROR: tensorflow 2.2.0rc3 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [0]:
!pip install --upgrade grpcio >> /dev/null

In [0]:
!pip install tqdm  >> /dev/null

In [0]:
!pip install bert-for-tf2 >> /dev/null

In [0]:
!pip install sentencepiece >> /dev/null

In [0]:
import os
import math
import datetime

from tqdm import tqdm

import pandas as pd
import pandas.util.testing as tm
import numpy as np

import tensorflow as tf
from tensorflow import keras

import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib import rc

from sklearn.metrics import confusion_matrix, classification_report

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

**LOAD DATASET INTO THE MODEL**

In [0]:
!gdown --id 1HEReqaaDMvpzeqJoaJHGniyN3gdEkNXs --output train.csv
!gdown --id 1DreaRLhJ-uInSoyhd0vxvK-cua_DaPYZ --output test.csv

Downloading...
From: https://drive.google.com/uc?id=1HEReqaaDMvpzeqJoaJHGniyN3gdEkNXs
To: /content/train.csv
100% 29.8k/29.8k [00:00<00:00, 49.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1DreaRLhJ-uInSoyhd0vxvK-cua_DaPYZ
To: /content/test.csv
100% 7.62k/7.62k [00:00<00:00, 6.65MB/s]


In [0]:
train = pd.read_csv("train.csv")

test = pd.read_csv("test.csv")

In [0]:
train.head()

,text,intents
0,any ideas why java plugin takes so long to load?,System Specification
1,java 1.5 loads _much_ faster,System Specification
2,how can i get 1.5 is there a .deb somewhere?,System Specification
3,is this blackdown or sun?,System Specification
4,did you install just the jre?,Settings


**INTENT RECOGONITION WITH BERT**

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2020-05-03 18:13:17--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.120.128, 2607:f8b0:4001:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.120.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   100MB/s    in 3.9s    

2020-05-03 18:13:21 (100 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [0]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [0]:
os.makedirs("model", exist_ok=True)

In [0]:
!mv uncased_L-12_H-768_A-12/ model

mv: cannot stat 'uncased_L-12_H-768_A-12/': No such file or directory


In [0]:
bert_model_name="uncased_L-12_H-768_A-12"

bert_ckpt_dir = os.path.join("model/", bert_model_name)
bert_ckpt_file = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

**PREPROCESSING**

In [0]:
class IntentDetectionData:
  DATA_COLUMN = "text"
  LABEL_COLUMN = "intents"

  def __init__(self, train, test, tokenizer: FullTokenizer, classes, max_seq_len=192):
    self.tokenizer = tokenizer
    self.max_seq_len = 0
    self.classes = classes
    
    train, test = map(lambda df: df.reindex(df[IntentDetectionData.DATA_COLUMN].str.len().sort_values().index), [train, test])
    
    ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self._prepare, [train, test])

    print("max seq_len", self.max_seq_len)
    self.max_seq_len = min(self.max_seq_len, max_seq_len)
    self.train_x, self.test_x = map(self._pad, [self.train_x, self.test_x])

  def _prepare(self, df):
    x, y = [], []
    
    for _, row in tqdm(df.iterrows()):
      text, label = row[IntentDetectionData.DATA_COLUMN], row[IntentDetectionData.LABEL_COLUMN]
      tokens = self.tokenizer.tokenize(text)
      tokens = ["[CLS]"] + tokens + ["[SEP]"]
      token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
      self.max_seq_len = max(self.max_seq_len, len(token_ids))
      x.append(token_ids)
      y.append(self.classes.index(label))

    return np.array(x), np.array(y)

  def _pad(self, ids):
    x = []
    for input_ids in ids:
      input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
      input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
      x.append(np.array(input_ids))
    return np.array(x)

In [0]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))

In [0]:
tokenizer.tokenize("did you install just the jre?")

['did', 'you', 'install', 'just', 'the', 'jr', '##e', '?']

In [0]:
tokens = tokenizer.tokenize("did you install just the jre?")
tokenizer.convert_tokens_to_ids(tokens)

[2106, 2017, 16500, 2074, 1996, 3781, 2063, 1029]

**FINE TUNING**

In [0]:
def create_model(max_seq_len, bert_ckpt_file):

  with tf.io.gfile.GFile(bert_config_file, "r") as reader:
      bc = StockBertConfig.from_json_string(reader.read())
      bert_params = map_stock_config_to_params(bc)
      bert_params.adapter_size = None
      bert = BertModelLayer.from_params(bert_params, name="bert")
        
  input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
  bert_output = bert(input_ids)

  print("bert shape", bert_output.shape)

  cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
  cls_out = keras.layers.Dropout(0.5)(cls_out)
  logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
  logits = keras.layers.Dropout(0.5)(logits)
  logits = keras.layers.Dense(units=len(classes), activation="softmax")(logits)

  model = keras.Model(inputs=input_ids, outputs=logits)
  model.build(input_shape=(None, max_seq_len))

  load_stock_weights(bert, bert_ckpt_file)
        
  return model

**TRAINING**

In [0]:
classes = train.intents.unique().tolist()

data = IntentDetectionData(train, test, tokenizer, classes, max_seq_len=128)

444it [00:00, 2453.56it/s]
114it [00:00, 2570.27it/s]

max seq_len 105


In [0]:
model = create_model(data.max_seq_len, bert_ckpt_file)

bert shape (None, 105, 768)
Done loading 196 BERT weights from: model/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7f16274845f8> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 105)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 105, 768)          108890112 
_________________________________________________________________
lambda (Lambda)              (None, 768)               0         
_________________________________________________________________
dropout (Dropout)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 768)               590592    
_________________________________________________________________
dropout_1 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 3076  

In [0]:
model.compile(
  optimizer=keras.optimizers.Adam(1e-5),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

In [0]:
log_dir = "log/intent_detection/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%s")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

history = model.fit(
  x=data.train_x, 
  y=data.train_y,
  validation_split=0.1,
  batch_size=16,
  shuffle=True,
  epochs=10,
  callbacks=[tensorboard_callback]
)

Train on 399 samples, validate on 45 samples
Epoch 1/10
399/399 [==============================] - 50s 124ms/sample - loss: 1.2826 - acc: 0.4160 - val_loss: 1.4357 - val_acc: 0.2889
Epoch 2/10
399/399 [==============================] - 28s 70ms/sample - loss: 1.2707 - acc: 0.4486 - val_loss: 1.4075 - val_acc: 0.2889
Epoch 3/10
399/399 [==============================] - 28s 70ms/sample - loss: 1.2334 - acc: 0.4837 - val_loss: 1.3282 - val_acc: 0.2889
Epoch 4/10
399/399 [==============================] - 28s 70ms/sample - loss: 1.2581 - acc: 0.4436 - val_loss: 1.2052 - val_acc: 0.4000
Epoch 5/10
399/399 [==============================] - 28s 70ms/sample - loss: 1.2273 - acc: 0.4787 - val_loss: 1.2977 - val_acc: 0.3333
Epoch 6/10
399/399 [==============================] - 28s 70ms/sample - loss: 1.1912 - acc: 0.5338 - val_loss: 1.2120 - val_acc: 0.4667
Epoch 7/10
399/399 [==============================] - 28s 70ms/sample - loss: 1.1654 - acc: 0.5539 - val_loss: 1.0887 - val_acc: 0.6667
Ep

In [0]:
_,test_acc = model.evaluate(data.test_x,data.test_y)
test_acc

114/114 [==============================] - 2s 18ms/sample - loss: 1.1340 - acc: 0.6140


0.61403507

In [0]:
y_pred = model.predict(data.test_x).argmax(axis=-1)

In [0]:
sentences = [
  "What's the best Wireframing tool?",
  "did you install just the jre?",
  "I use IBM's 1.4.2",
  
]

tokens = map(tokenizer.tokenize, sentences)
tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], tokens)
token_ids = list(map(tokenizer.convert_tokens_to_ids, tokens))

token_ids = map(
  lambda tids: tids +[0]*(data.max_seq_len-len(tids)),
  token_ids
)
token_ids = np.array(list(token_ids))

predictions = model.predict(token_ids).argmax(axis=-1)
res=[]
for text, label in zip(sentences, predictions):
  print("text:", text, "\nintent:", classes[label])
  temp=text,classes[label]
  res.append(temp)
  print()

text: What's the best Wireframing tool? 
intent: System Specification

text: did you install just the jre? 
intent: Settings

text: I use IBM's 1.4.2 
intent: System Specification



In [0]:
print(res)

NameError: ignored

# **GOBOT model using deeppavlov**

# Download these packages

In [0]:
#1 
!pip install fasttext

     |████████████████████████████████| 71kB 2.7MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3021175 sha256=4028ea0c2563e4562e4b8c90a83e84c84e00e4da4b4480c0f10e92a370293a42
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [0]:
#2
pip install deeppavlov

     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 7.3MB 7.9MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
     |████████████████████████████████| 8.0MB 27.0MB/s 
     |████████████████████████████████| 1.5MB 43.0MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 20.1MB 111kB/s 
     |████████████████████████████████| 665kB 39.0MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 10.4MB 39.0MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 6.7MB 41.4MB/s 
     |████████████████████████████████| 2.1MB 36.2MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |█████

In [0]:
#3
pip uninstall tensorflow

Uninstalling tensorflow-1.13.2:
  Would remove:
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.13.2.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.13.2


In [0]:
#4
pip install tensorflow==1.13.2

  Using cached https://files.pythonhosted.org/packages/db/d3/651f95288a6cd9094f7411cdd90ef12a3d01a268009e0e3cd66b5c8d65bd/tensorflow-1.13.2-cp36-cp36m-manylinux1_x86_64.whl


# Run the below code and the dataset of simple dstc will be downloaded. But for this project AskUbuntu Corpus is used so copy the dataset from the provided file and replace the dataset without changing the file name.

# After copying the dataset run the below command one more time.The dataset count will be changed to 50,25,25.

In [0]:
#5
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader
data=SimpleDSTC2DatasetReader.read(
    data_path="my data"
)

2020-05-03 18:41:24.608 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from my data/simple-dstc2-trn.json]
2020-05-03 18:41:24.613 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from my data/simple-dstc2-val.json]
2020-05-03 18:41:24.615 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from my data/simple-dstc2-tst.json]
2020-05-03 18:41:24.618 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 50 samples in train split.
2020-05-03 18:41:24.619 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 50 samples in valid split.
2020-05-03 18:41:24.622 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 50 samples in test split.


In [0]:
#6
!head -n 10 'my data/simple-dstc2-trn.json'

[
  [
    {
      "speaker": 1,
      "text": "hi"
    },
    {
      "speaker": 2,
      "text": "hi,what can i help you with",
      "slot":[],


In [0]:
#7
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator
iterator=DialogDatasetIterator(data)

In [0]:
#8
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1,data_type="train"):
  turn_x,turn_y=dialog
  print("User utterances:\n----------------\n")
  pprint(turn_x[0], indent=4)
  print("\nSystem responses:\n-----------------\n")
  pprint(turn_y[0], indent=4)
  break;

User utterances:
----------------

[   {'prev_resp_act': None, 'text': 'hi'},
    {'prev_resp_act': 'welcomemsg', 'text': 'Software to read a QR code?'},
    {   'db_result': {'problem': 'QR code'},
        'prev_resp_act': 'api_call',
        'text': 'Software to read a QR code?'},
    {'prev_resp_act': 'software_recommendation_qr_code', 'text': 'ohh..'},
    {'prev_resp_act': 'softwarerecommendation', 'text': 'thank you good bye'}]

System responses:
-----------------

[   {'act': 'welcomemsg', 'text': 'hi,what can i help you with'},
    {'act': 'api_call', 'text': 'api_call problem="QR code"'},
    {   'act': 'software_recommendation_qr_code',
        'text': '<p><strong>FOR QR CODE</strong></p><p>Install <a '
                'href="https://launchpad.net/ubuntu/+source/libdecodeqr-examples" '
                'rel="nofollow noreferrer">libdecodeqr-examples</a> from the '
                'repositories,</p><p>Use the program, '
                '<code>libdecodeqr-simpletest</code> provi

### **Creating the database**

In [0]:
from deeppavlov.core.data.sqlite_database import Sqlite3Database

database = Sqlite3Database(primary_keys=["problem"],
                           save_path="my_bot/db.sqlite")

2020-05-03 18:46:46.458 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2020-05-03 18:46:46.460 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 70: Initializing empty database on /content/my_bot/db.sqlite.


In [0]:
db_results = []

for dialog in iterator.gen_batches(batch_size=1, data_type='all'):
    turns_x, turns_y = dialog
    db_results.extend(x['db_result'] for x in turns_x[0] if x.get('db_result'))

print(f"Adding {len(db_results)} items.")
if db_results:
    database.fit(db_results)

2020-05-03 18:46:52.416 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 130: Created table with keys {'problem': 'text'}.


Adding 30 items.


So Database have been created and the primary key is set as "Problem" because it can store new problems and if the problem is already present in the dataset then there will be no change. In this way the bot can train itself with when new problem arises.

In [0]:
database([{'problem':'Mind Mapping'}])

[[{'problem': 'Mind Mapping'}]]

## **Creating Slotfiller**

Run the code below and a dstc2 json file will be uploaded.From the provided dstc2 json copy the data and replace them. Don't run the command after copying the data.

In [0]:
from deeppavlov.download import download_decompress

download_decompress(url='http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz',
                    download_path='my_bot/slotfill')

2020-05-03 18:35:08.704 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz to my_bot/slotfill/dstc_slot_vals.tar.gz
100%|██████████| 1.62k/1.62k [00:00<00:00, 2.46MB/s]
2020-05-03 18:35:08.710 INFO in 'deeppavlov.core.data.utils'['utils'] at line 242: Extracting my_bot/slotfill/dstc_slot_vals.tar.gz archive into my_bot/slotfill


In [0]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

slotfill_config = read_json(configs.ner.slotfill_simple_dstc2_raw)

In [0]:
slotfill_config['metadata']['variables']['DATA_PATH'] = 'my data'
slotfill_config['metadata']['variables']['SLOT_VALS_PATH'] = 'my_bot/slotfill/dstc_slot_vals.json'

In [0]:
from deeppavlov import evaluate_model

slotfill = evaluate_model(slotfill_config);

2020-05-03 18:47:05.140 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/my data/simple-dstc2-trn.json]
2020-05-03 18:47:05.142 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/my data/simple-dstc2-val.json]
2020-05-03 18:47:05.144 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/my data/simple-dstc2-tst.json]
2020-05-03 18:47:05.149 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 50 samples in train split.
2020-05-03 18:47:05.150 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 50 samples in valid split.
2020-05-03 18:47:05.151 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 50 samples in test split.


{"valid": {"eval_examples_count": 8, "metrics": {"slots_accuracy": 0.5}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 8, "metrics": {"slots_accuracy": 0.5}, "time_spent": "0:00:01"}}


# The slot accuracy is found to be 0.5

In [0]:
from deeppavlov import build_model

slotfill = build_model(slotfill_config)

By running the below command the slotfiller will display the problem i.e the intent of the sentence. Slotfiller  can read sentence even it is written in lower case or upper case.
After running the below command try to change the below sentence as 


```
slotfill(['Software to read a QR CODE?'])
```



In [0]:
slotfill(['Software to read a qr code?'])

[{'problem': 'QR code'}]

In [0]:
import json

json.dump(slotfill_config, open('my_bot/slotfill_config.json', 'wt'))

### **Developing the Model**

In [0]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_simple_dstc2)

The below command isto set which type of embedder to be used. When it is set to "None" then one-hot vector is loaded. We can set Fasttext,elmo,glove embedders also.

In [0]:
gobot_config['chainer']['pipe'][-1]['embedder'] = None

In [0]:
gobot_config['chainer']['pipe'][-1]['database'] = {
    'class_name': 'sqlite_database',
    'primary_keys': ["problem"],
    'save_path': 'my_bot/db.sqlite'
}

In [0]:
gobot_config['chainer']['pipe'][-1]['slot_filler']['config_path'] = 'my_bot/slotfill_config.json'

In [0]:
gobot_config['chainer']['pipe'][-1]['tracker']['slot_names'] = ['problem']

In [0]:
gobot_config['chainer']['pipe'][-1]['template_type'] = 'DefaultTemplate'
gobot_config['chainer']['pipe'][-1]['template_path'] = 'my data/simple-dstc2-templates.txt'

In [0]:
gobot_config['metadata']['variables']['DATA_PATH'] = 'my data'
gobot_config['metadata']['variables']['MODEL_PATH'] = 'my_bot'

In [0]:
from deeppavlov import train_model

gobot_config['train']['batch_size'] = 4 
gobot_config['train']['max_batches'] = 250 
gobot_config['train']['log_on_k_batches'] = 20
gobot_config['train']['val_every_n_batches'] = 40 
gobot_config['train']['log_every_n_batches'] = 40 

train_model(gobot_config);

2020-05-03 18:47:40.633 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/my data/simple-dstc2-trn.json]
2020-05-03 18:47:40.636 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/my data/simple-dstc2-val.json]
2020-05-03 18:47:40.639 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/my data/simple-dstc2-tst.json]
2020-05-03 18:47:40.641 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 50 samples in train split.
2020-05-03 18:47:40.644 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 50 samples in valid split.
2020-05-03 18:47:40.648 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 50 samples in test split.
2020-05-03 18:47:40.651 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at

{"valid": {"eval_examples_count": 10, "metrics": {"per_item_dialog_accuracy": 0.2}, "time_spent": "0:00:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}


2020-05-03 18:47:44.984 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 206: Improved best per_item_dialog_accuracy of 0.76
2020-05-03 18:47:44.985 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 208: Saving model
2020-05-03 18:47:44.986 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /content/my_bot/model]


{"train": {"eval_examples_count": 10, "metrics": {"per_item_dialog_accuracy": 0.76}, "time_spent": "0:00:03", "epochs_done": 13, "batches_seen": 40, "train_examples_seen": 134, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.5142218430992216}}


2020-05-03 18:47:45.149 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 711: [saving parameters to /content/my_bot/model.json]


...

INFO:tensorflow:Restoring parameters from /content/my_bot/model


2020-05-03 18:48:00.610 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/my_bot/word.dict]
2020-05-03 18:48:00.612 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2020-05-03 18:48:00.615 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /content/my_bot/db.sqlite.


{"valid": {"eval_examples_count": 10, "metrics": {"per_item_dialog_accuracy": 0.76}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 10, "metrics": {"per_item_dialog_accuracy": 0.76}, "time_spent": "0:00:01"}}


2020-05-03 18:48:01.306 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 164: [loading templates from /content/my data/simple-dstc2-templates.txt]
2020-05-03 18:48:01.308 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 167: 9 templates loaded.
2020-05-03 18:48:01.310 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 215: Calculated input size for `GoalOrientedBotNetwork` is 57
2020-05-03 18:48:02.394 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 253: [initializing `GoalOrientedBot` from saved]
2020-05-03 18:48:02.395 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 717: [loading parameters from /content/my_bot/model.json]
2020-05-03 18:48:02.402 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/my_bot/model]


INFO:tensorflow:Restoring parameters from /content/my_bot/model


In [0]:
from deeppavlov import evaluate_model

evaluate_model(gobot_config);

2020-05-03 18:48:02.477 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/my data/simple-dstc2-trn.json]
2020-05-03 18:48:02.479 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/my data/simple-dstc2-val.json]
2020-05-03 18:48:02.483 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 290: [loading dialogs from /content/my data/simple-dstc2-tst.json]
2020-05-03 18:48:02.487 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 282: There are 50 samples in train split.
2020-05-03 18:48:02.488 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: There are 50 samples in valid split.
2020-05-03 18:48:02.490 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: There are 50 samples in test split.
2020-05-03 18:48:02.495 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at

INFO:tensorflow:Restoring parameters from /content/my_bot/model
{"valid": {"eval_examples_count": 10, "metrics": {"per_item_dialog_accuracy": 0.76}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 10, "metrics": {"per_item_dialog_accuracy": 0.76}, "time_spent": "0:00:01"}}


# The accuracy of per dialogue is found to be 0.76

In [0]:
from deeppavlov import build_model

bot = build_model(gobot_config)

2020-05-03 18:48:04.613 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/my_bot/word.dict]
2020-05-03 18:48:04.615 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2020-05-03 18:48:04.617 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /content/my_bot/db.sqlite.
2020-05-03 18:48:05.108 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 164: [loading templates from /content/my data/simple-dstc2-templates.txt]
2020-05-03 18:48:05.109 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 167: 9 templates loaded.
2020-05-03 18:48:05.110 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 215: Calculated input size for `GoalOrientedBotNetwork` is 57
2020-05-03 18:48:06.254 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 253: [initial

INFO:tensorflow:Restoring parameters from /content/my_bot/model


In [0]:
bot(['hi'])

['Hi,what can i help you with']

# Can ask queries like 
# 1.What software can I use to view epub documents?
# 2.Which PDF Viewer would you recommend?
# 3.What IDEs are available for Ubuntu?
# 4.What's the best Mind Mapping Software?
# 5.Software to read a QR code?

In [0]:
a=bot(['Software to read a qr code?'])

2020-05-03 18:48:06.433 INFO in 'deeppavlov.models.go_bot.tracker'['tracker'] at line 210: Made api_call with {'problem': 'QR code'}, got 1 results.


In [0]:
print(a)

['The Solution for QR code is <p><strong>FOR QR CODE</strong></p><p>Install <a href=\\"https://launchpad.net/ubuntu/+source/libdecodeqr-examples\\" rel=\\"nofollow noreferrer\\">libdecodeqr-examples</a> from the repositories,</p><p>Use the program, <code>libdecodeqr-simpletest</code> provided by the package to read your QR encoded image</p><p>EXAMPLE:</p><p>running this command to read one of the test samples in the examples img folder</p><pre><code>libdecodeqr-simpletest /usr/share/doc/libdecodeqr-examples/img/debian.or.jp.qr.jpg</code></pre><p>Gives the decoded output in the console.</p><pre><code>libdecodeqr version 0.9.3 ($Rev: 42 $)STATUS=2080http://www.debian.or.jp MEBKM:TITLE:DebianJP;URL:http\\\\://www.debian.or.jp;;</code></pre><p>as can be seen a url was encoded in the file</p><p>There is also a sample application that works with your webcam called libdecodeqr-webcamyou need a webcam to use this, I have not tested it.</p><p>IF you would like to generate QR code you can instal

The sentence Solution for "**QR code**".
QR code is the intent which retrived from the slot and is replyed to the user.

In [0]:
bot(['oh..'])

['You can try this recommendation with no worries']

In [0]:
bot(['thank you good bye'])

['You are welcome!']

In [0]:
bot.reset()

In [0]:
utterance = ""
while utterance != 'exit':
    print(">> " + bot([utterance])[0])
    utterance = input(':: ')

# Therefore the intent is found.RNN and one hot encoder has been implemented.

In [0]:
!pip install flask
!pip install flask-ngrok

In [0]:
print(a)

['The Solution for QR code is <p><strong>FOR QR CODE</strong></p><p>Install <a href=\\"https://launchpad.net/ubuntu/+source/libdecodeqr-examples\\" rel=\\"nofollow noreferrer\\">libdecodeqr-examples</a> from the repositories,</p><p>Use the program, <code>libdecodeqr-simpletest</code> provided by the package to read your QR encoded image</p><p>EXAMPLE:</p><p>running this command to read one of the test samples in the examples img folder</p><pre><code>libdecodeqr-simpletest /usr/share/doc/libdecodeqr-examples/img/debian.or.jp.qr.jpg</code></pre><p>Gives the decoded output in the console.</p><pre><code>libdecodeqr version 0.9.3 ($Rev: 42 $)STATUS=2080http://www.debian.or.jp MEBKM:TITLE:DebianJP;URL:http\\\\://www.debian.or.jp;;</code></pre><p>as can be seen a url was encoded in the file</p><p>There is also a sample application that works with your webcam called libdecodeqr-webcamyou need a webcam to use this, I have not tested it.</p><p>IF you would like to generate QR code you can instal

Create a folder named as "templates" and add the index.html file to that folder and run the below command.

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask,render_template

app=Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
  return render_template("index.html",answer=a)

if __name__=="__main__":
  app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://95dfd96b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [03/May/2020 18:50:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 18:50:06] "GET /favicon.ico HTTP/1.1" 404 -
